In [ ]:
import socket
import pandas as pd

SERVER_IP = "127.0.0.1"
SERVER_PORT = 4000

def display_menu():
    print("\n\t\t\t******WELCOME TO TRAIN BOOKING SYSTEM******")
    print("Choose one of the following options:")
    print("1. LogIn")
    print("2. Register")
    print("3. Exit")
    print("4. Book Ticket")
    print("5. Cancel Ticket")
    print("6. Train Status")
    return int(input("Your choice: "))

def send_data(client_socket, data):
    try:
        client_socket.sendall(data.encode())
    except Exception as e:
        print(f"Error sending data: {e}")
        return False
    return True

def recv_data(client_socket, buffer_size=1024):
    try:
        data = client_socket.recv(buffer_size).decode()
    except Exception as e:
        print(f"Error receiving data: {e}")
        return None
    return data

def main():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((SERVER_IP, SERVER_PORT))

    while True:
        choice = display_menu()
        if not send_data(client_socket, str(choice)):
            break

        if choice == 1:
            name = input("Enter name: ")
            code = input("Enter code: ")
            login_details = f"{name},{code}"
            if not send_data(client_socket, login_details):
                break
            response = recv_data(client_socket)
            if response == "1":
                print("Login successful.")
            else:
                print("Login failed.")
        elif choice == 2:
            name = input("Enter name: ")
            code = input("Enter code: ")
            address = input("Enter address: ")
            phone = input("Enter phone: ")
            registration_details = f"{name},{code},{address},{phone}"
            if not send_data(client_socket, registration_details):
                break
            response = recv_data(client_socket)
            if response == "1":
                print("Registration successful.")
            else:
                print("Registration failed.")
        elif choice == 3:
            print("Exiting.")
            break
        elif choice == 4:
            df = pd.read_csv('trains.csv')
            print(df)
            train_id = input("Enter train ID to book: ")
            seats = input("Enter number of seats to book: ")
            if not send_data(client_socket, train_id):
                break
            if not send_data(client_socket, seats):
                break
            ticket_details = recv_data(client_socket)
            if ticket_details:
                print(f"Ticket details: {ticket_details}")
        elif choice == 5:
            prn = input("Enter PRN to cancel ticket: ")
            if not send_data(client_socket, prn):
                break
            response = recv_data(client_socket)
            if response == "1":
                print("Ticket cancellation successful.")
            else:
                print("Ticket cancellation failed.")
        elif choice == 6:
            train_id = input("Enter train ID to check status: ")
            if not send_data(client_socket, train_id):
                break
            train_status = recv_data(client_socket)
            if train_status:
                print(train_status)
            else:
                print("Error fetching train status.")
    
    client_socket.close()

if __name__ == "__main__":
    main()
